This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = '2yVfYYaiqNwsBjRcfQq-'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: https://requests.readthedocs.io/en/master/

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests


In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X'
start_date = '2017-01-01'
end_date = '2018-01-01'

params = {
    'api_key' : API_KEY,
    'start_date' : start_date,
    'end_date' : end_date
}

r = requests.get(url, params=params)

In [11]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
r.json()

{'dataset': {'id': 10095370,
  'dataset_code': 'AFX_X',
  'database_code': 'FSE',
  'name': 'Carl Zeiss Meditec (AFX_X)',
  'description': 'Stock Prices for Carl Zeiss Meditec (2020-05-08) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704',
  'refreshed_at': '2020-05-22T22:16:19.751Z',
  'newest_available_date': '2020-05-22',
  'oldest_available_date': '2000-06-07',
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'frequency': 'daily',
  'type': 'Time Series',
  'premium': False,
  'limit': None,
  'transform': None,
  'column_index': None,
  'start_date': '2020-01-01',
  'end_date': '2020-01-03',
  'data': [['2020-01-03',
    112.8,
    113.4,
    112.0,
    113.4,
    None,
    82304.0,
    9289987.0,
    None,
    None,
    None],
   ['2020-01-02',
    114.0,
    114.9,
    112.7,
    113.8,
    Non

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD)

In [4]:
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X'
start_date = '2017-01-01'
end_date = '2018-01-01'

params = {
    'api_key' : API_KEY,
    'start_date' : start_date,
    'end_date' : end_date
}

r = requests.get(url, params=params)

2. Convert the returned JSON object into a Python dictionary.

In [5]:
data = r.json()

print(data['dataset']['column_names'])

['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']


3. Calculate what the highest and lowest opening prices were for the stock in this period.

In [18]:
open_prices = [e[1] for e in data['dataset']['data'] if e[1] is not None]
print(f'Lowest opening price: {min(open_prices)}, highest openining price: {max(open_prices)}')

Lowest opening price: 34.0, highest openining price: 53.11


4. What was the largest change in any one day (based on High and Low price)?

In [24]:
largest_change = -float('inf')

for day in data['dataset']['data']:
    if day[2] and day[3]:
        if day[2] - day[3] > largest_change: largest_change = day[2] - day[3]
# largest_change = [day[2] - day[3] for day in data['dataset']['data'] if (day[2] and day[3])]
largest_day_to_day_change = max([abs(days[i][4] - days[i+1][4]) for i in range(len(days)-1)])
print(f'Largest change in one day between Hign and Low prices: {round(largest_change,2)}')

Largest change in one day between Hign and Low prices: 2.81


5. What was the largest change between any two days (based on Closing Price)?

In [36]:
largest_two_days_change = -float('inf')
days = data['dataset']['data']

for i in range(len(days)-2):
    if days[i][4] and days[i+1][4]:
        candidate1 = abs(days[i][4] - days[i+1][4])
    if days[i][4] and days[i+2][4]:
        candidate2 = abs(days[i][4] - days[i+2][4])
    if max(candidate1, candidate2) > largest_two_days_change: 
        largest_two_days_change = max(candidate1, candidate2)

print(f'Largest change in closing price between any two days: {round(largest_two_days_change,2)}')

Largest change in closing price between any two days: 3.15


6. What was the average daily trading volume during this year?

In [49]:
daily_traded_units = [e[6] for e in days if e[6] is not None]
ADTV = sum(daily_traded_units)/len(daily_traded_units)
print(f'Average daily trading volume: {ADTV}')

Average daily trading volume: 89124.33725490196


7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [55]:
median_volume = sorted(daily_traded_units)[int(len(sorted(daily_traded_units))/2)]
print(f'Median trading volume: {median_volume}')

Median trading volume: 76286.0
